In [4]:
import numpy as np
import pandas as pd
import os
import re
from tqdm import tqdm
import requests
from io import StringIO
from os import listdir
from os.path import isfile, join
import datetime as dt
from datetime import datetime, timedelta

import gridstatus

In [5]:
def listdir_nohidden(path):
    for f in os.listdir(path):
        if not f.startswith('.'):
            yield f



In [6]:
FILE_PATH = "/Users/evanlipton/Dropbox (YSE)/Solar_ElectricData/Data"

In [7]:
ISOS = ["SPP"]

In [8]:
ISO_files = {}
ISO_data = {}
ISO_final = {}

#load in
for ISO in ISOS:
    ISO_files[ISO] = listdir_nohidden(f"{FILE_PATH}/{ISO}/old")
    ISO_data[ISO] = {}
    for file in tqdm(ISO_files[ISO]):
        compiled = re.compile(re.escape(f"{ISO}_"), re.IGNORECASE)

        ISO_data[ISO][compiled.sub("", file).replace(".csv","")] = pd.read_csv(f"{FILE_PATH}/{ISO}/old/{file}")

13it [00:02,  6.14it/s]


In [9]:
#add merge datetime
titles = {}
for key in ISO_data:
    print(key)
    titles[key]= []
    for k in ISO_data[key]:
        try:
            ISO_data[key][k]["time_tomerge"] = pd.to_datetime(ISO_data[key][k].iloc[:,0], utc=True)
        except:
            ISO_data[key][k] = ISO_data[key][k][ISO_data[key][k].index % 24 != 0]
            ISO_data[key][k]["time_tomerge"] = pd.to_datetime(ISO_data[key][k].iloc[:,0], utc=True)
            print(type(ISO_data[key][k]["time_tomerge"]))

SPP


In [10]:
ISO_final = {}
def clean_final(ISO: str, df: str):
    ISO_data[ISO][df].columns = ISO_data[ISO][df].columns.str.lower()
    no_date = [col for col in ISO_data[ISO][df].columns if "date" not in col]
    ISO_data[ISO][df] = ISO_data[ISO][df][no_date]
    try:
        ISO_data[ISO][df] = ISO_data[ISO][df].drop("unnamed: 0")
        ISO_data[ISO][df] = ISO_data[ISO][df].drop("hour")
    except:
        pass
    ISO_data[ISO][df] = ISO_data[ISO][df].sort_values(by="time_tomerge")
    ISO_data[ISO][df] = ISO_data[ISO][df].set_index("time_tomerge")
    ISO_final[ISO] = ISO_data[ISO][df]
    for column in ISO_final[ISO].columns:
        if str(ISO).lower() in column:
            ISO_final[ISO] = ISO_final[ISO].rename(columns={column: column.replace(f"{str(ISO).lower()}_","")})
    print(ISO_final[ISO])

In [11]:
for iso in ISOS:
    clean_final(iso, "final")

                           losses_rt  losses_rt.1  energy_rt  energy_da  \
time_tomerge                                                              
2014-01-01 06:00:00+00:00        NaN          NaN        NaN        NaN   
2014-01-01 07:00:00+00:00        NaN          NaN        NaN        NaN   
2014-01-01 08:00:00+00:00        NaN          NaN        NaN        NaN   
2014-01-01 09:00:00+00:00        NaN          NaN        NaN        NaN   
2014-01-01 10:00:00+00:00        NaN          NaN        NaN        NaN   
...                              ...          ...        ...        ...   
2019-01-01 01:00:00+00:00   -0.13040     -0.13040    12.5285    17.4804   
2019-01-01 02:00:00+00:00   -0.14715     -0.14715    12.7049    16.7689   
2019-01-01 03:00:00+00:00   -0.15080     -0.15080    13.1140    15.4343   
2019-01-01 04:00:00+00:00   -0.36170     -0.36170    20.7879    15.7768   
2019-01-01 05:00:00+00:00   -0.09535     -0.09535     9.2530    14.6722   

                        

In [13]:
ISO_final[ISO].columns


Index(['losses_rt', 'losses_rt.1', 'energy_rt', 'energy_da', 'congestion_rt',
       'congestion_da', 'exchange', 'gas_price', 'gen_coal', 'gen_oil',
       'gen_hydro', 'gen_gas', 'gen_nuclear', 'gen_solar', 'gen_waste',
       'gen_wind', 'gen_wasteheat', 'gen_other', 'load', 'load.1'],
      dtype='object')

In [15]:
FILE_PATH = "/Users/evanlipton/Dropbox (YSE)/Solar_ElectricData/Data/SPP/new"
energy_rt = pd.read_csv(f"{FILE_PATH}/combined/energy_rt.csv")
print(energy_rt)

         Unnamed: 0        Date  Settlement Location Name  \
0              2638  2019/01/01       WFEC_ANA_COMB_CYC_4   
1              1866  2019/01/01         SECI.UCU.GRAYWIND   
2              1867  2019/01/01         SECI.UCU.GRAYWIND   
3              1868  2019/01/01         SECI.UCU.GRAYWIND   
4              1869  2019/01/01                  SECIABBK   
...             ...         ...                       ...   
4703596      106493  2023/01/01             MPSLAKE_RDUN6   
4703597      106494  2023/01/01             MPSLAKE_RDUN7   
4703598      106495  2023/01/01             MPSLAKE_RDUN7   
4703599      106497  2023/01/01              MPSNEVADAUN1   
4703600      108754  2023/01/01              WR_MOPEP_MPS   

                           PNODE Name  Price Type     HE01     HE02     HE03  \
0                  WFECANADARKOUN4_RA         MCC      NaN      NaN      NaN   
1        SECIHAGGAR1UNWINDFARM_MPS_RA         LMP      NaN      NaN      NaN   
2        SECIHAGGAR1UNWINDF

In [18]:
print(list(np.unique(energy_rt[" Settlement Location Name"])))

['AEC', 'AECC_CSWS', 'AECC_ELKINS', 'AECC_FITZHUGH', 'AECC_FLTCREEK', 'AECC_FULTON', 'AECC_HYDRO13', 'AECC_JWTURK', 'AECI', 'AEPM_CSWS', 'ALTW', 'AMRN', 'ANTELOPE', 'BBA', 'BCA', 'BEPM_RM_NPPD', 'BEPM_TS_NPPD', 'BLKW', 'BLUECANYON2', 'BLUECANYON5', 'BLUECANYON6', 'BRAZ', 'BUBA', 'BUFFALO_DUNES_WIND', 'CANADIAN_HILLS_1', 'CANADIAN_HILLS_2', 'CANADIAN_HILLS_3', 'CANADIAN_HILLS_4', 'CANADIAN_HILLS_5', 'CANEY_RIVER_WINDPOWER', 'CASS_CO_1', 'CASS_CO_2', 'CLEC', 'COFFEYVILLE_7', 'COWP', 'COWP_GEN1', 'COWP_GEN2', 'CPLE', 'CPLW', 'CROSSROADS1', 'CROSSROADS2', 'CROSSROADS3', 'CROSSROADS4', 'CRTH_SPA', 'CSWARSENALHILL5', 'CSWCOMANCHE1', 'CSWDOLET1', 'CSWELKCITY', 'CSWETTURK', 'CSWFLINTCREEK1', 'CSWJLSTALL', 'CSWKNOXLEE2', 'CSWKNOXLEE3', 'CSWKNOXLEE4', 'CSWKNOXLEE5', 'CSWLIEBERMAN2', 'CSWLIEBERMAN3', 'CSWLIEBERMAN4', 'CSWLONESTAR1', 'CSWMAJESTICWIND', 'CSWMATTISON1', 'CSWMATTISON2', 'CSWMATTISON3', 'CSWMATTISON4', 'CSWMSTURK', 'CSWNARROWS1', 'CSWNORTHEASTERN1', 'CSWNORTHEASTERN2', 'CSWNORTHEASTER

In [20]:
print(list(np.unique(energy_rt[" PNODE Name"])))


['11HL_DDR', '29THSTREET_DDR', '47THWEBB_DDR', '53CACHE4_DDR', '72ELWD_DDR', '81GARN_DDR', '8ST2_DDR', 'AECI', 'AFTONTP_DDR', 'ARDMORE_DDR', 'BANORTH_DDR', 'BART_DDR', 'BELLEISLE_DDR', 'BIX111_DDR', 'BLKW', 'BLUCRCLT_DDR', 'BODLE_DDR', 'BONITA_DDR', 'BOYD_DDR', 'BRKRIDGE_DDR', 'BROOKRIDGE_DDR', 'CENTER_DDR', 'COULTER_DDR', 'COWP_LA', 'CSWDOLET1_H', 'CSWELKCITY_H', 'CSWKNOXLEE2_H', 'CSWKNOXLEE3_H', 'CSWKNOXLEE4_H', 'CSWLIEBERMAN2_H', 'CSWLONESTAR1_H', 'CSWMAJESTICWIND_H', 'CSWS.CORP.KIAPB1_H', 'CSWS.CORP.KIAPB2_H', 'CSWS.ELK2.ELKCITY2_H', 'CSWS.FPLP.RSES_H', 'CSWS.GREENCOUNTRY_2_H', 'CSWS.GREENCOUNTRY_3_H', 'CSWS.GREENCOUNTRY_H', 'CSWS.VOLT.0002_H', 'CSWS.VOLT.0025_H', 'CSWS.VOLT.0040_H', 'CSWS.VOLT.0058_H', 'CSWS.WTHR.WTHR_H', 'CSWSALBAUNALBA_DDR_GEN_RA', 'CSWSARSENALUN5_RA', 'CSWSCALPINEUNONETA_1_RA', 'CSWSCALPINEUNONETA_2_RA', 'CSWSCALPINEUNONETA_3_RA', 'CSWSCALPINEUNONETA_4_RA', 'CSWSCOMANCHEPLT1', 'CSWSDEMPSRDGUNDEMPSEY_WIND_RA', 'CSWSDOLET_PPUN1_RA', 'CSWSDOLET_PPUNOMPA_DOLET_RA',

In [21]:
print(energy_rt)

,Unnamed: 0,Date,Settlement Location Name,PNODE Name,Price Type,HE01,HE02,HE03,HE04,HE05,...,HE16,HE17,HE18,HE19,HE20,HE21,HE22,HE23,HE24,time_tomerge
0,2638,2019/01/01,WFEC_ANA_COMB_CYC_4,WFECANADARKOUN4_RA,MCC,NaN,NaN,NaN,NaN,NaN,...,16.8310,15.3952,0.2452,0.1323,0.1658,0.1363,0.1477,0.1608,0.1253,2019-01-01
1,1866,2019/01/01,SECI.UCU.GRAYWIND,SECIHAGGAR1UNWINDFARM_MPS_RA,LMP,NaN,NaN,NaN,NaN,NaN,...,16.8795,27.1616,85.9528,29.9915,32.3909,28.9934,30.3746,31.8390,38.1496,2019-01-01
2,1867,2019/01/01,SECI.UCU.GRAYWIND,SECIHAGGAR1UNWINDFARM_MPS_RA,MCC,NaN,NaN,NaN,NaN,NaN,...,-1.7649,-1.4075,0.4250,0.2297,0.2884,0.2376,0.2579,0.2800,0.3733,2019-01-01
3,1868,2019/01/01,SECI.UCU.GRAYWIND,SECIHAGGAR1UNWINDFARM_MPS_RA,MLC,NaN,NaN,NaN,NaN,NaN,...,-4.6640,-4.2580,-4.5614,-0.2297,0.4887,0.6538,0.7930,1.0306,1.3449,2019-01-01
4,1869,2019/01/01,SECIABBK,SECIROLLAUNABENGOA_RA,LMP,NaN,NaN,NaN,NaN,NaN,...,20.6605,31.2483,92.2376,31.2679,33.2991,30.0343,31.5381,32.9207,39.1818,2019-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4703596,106493,2023/01/01,MPSLAKE_RDUN6,MPSLAKE_RDUN6_RA,MLC,0.6254,0.5792,0.6086,0.6144,0.6241,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-01-01
4703597,106494,2023/01/01,MPSLAKE_RDUN7,MPSLAKE_RDUN7_RA,LMP,26.4815,24.7724,22.9408,19.8480,18.7161,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-01-01
4703598,106495,2023/01/01,MPSLAKE_RDUN7,MPSLAKE_RDUN7_RA,MCC,1.6617,1.9470,1.8527,1.4657,1.2316,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-01-01
4703599,106497,2023/01/01,MPSNEVADAUN1,MPSNEVADAUN1_RA,LMP,26.5294,25.5485,23.6322,20.7150,19.4349,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-01-01
